<a href="https://colab.research.google.com/github/konstantint/ai-auto-olympiad/blob/main/informatics/generic/ancient_runes_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

AI plays informatics olympiad with itself

# Initialization boilerplate

In [ ]:
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.4/155.4 kB 3.1 MB/s eta 0:00:00


In [ ]:
#@title Imports & api key loading
import google.generativeai as genai
import os
from google.colab import userdata
import ipywidgets as widgets # Import ipywidgets
from IPython.display import display # Import display to show widgets

# --- Configuration ---

# Fetch the API key from Colab secrets
try:
    api_key = userdata.get('GOOGLE_API_KEY')
    if not api_key:
        raise ValueError("API key not found. Please add it to Colab secrets under the name 'GOOGLE_API_KEY'.")
    genai.configure(api_key=api_key)
except Exception as e:
    print(f"Error configuring Gemini API: {e}")
    # You might want to exit or handle this error appropriately
    exit()


# Part 1: AI creates an olympiad task

First, the olympiad task author comes up with an olympiad task along with a set of tests for it.

In [ ]:
#@title Prompt
task_author_prompt = widgets.Textarea(
    value="""You are an experienced author of informatics olympiad problems.

Your task is to come up with a perfect programming task for the Baltics Olympiad in Informatics.
Informatics olympiad problems must have the following properties:
  - Their statement reads like a real-life situation or a story where some character needs to solve some
    kind of a problem they are facing.
  - The solution to the problem involves the use of algorithmic coding. Participants will write a program
    in Python that must read the input data via stdin and output the result via stdout.
  - The participants usually need to come up with a clever combination of one or more classical
    algorithms in order to solve the problem.
  - The problem must allow a range of solutions using various algorithmic techniques, with different
    algorithmic complexities (e.g. a brute force solution might be O(n^3) while a smart use of dynamic programming
    could allow a linear solution, or the like).
  - The optimal solution to the problem could be found, implemented and tested by an olympiad participant (who
    is experienced in competitive programming) within about 30-60 minutes.
  - The problem must be accompanied by a number of tests, where each test consists of a text file with problem input data
    and a text file with correct output.

You are trying to develop a new olympiad problem. Please, proceed step by step as follows:

# Step 1.

List the most common algorithms or algorithmic techniques that are usually involved in informatics olympiad problems.

# Step 2.

Using the list of algorithms produced in Step 1, come up with five potential problems which require a combination of two or three
algorithmics techniques listed in Step 1 to be solved.

# Step 3.

For each of the problems you produced in Step 2, verify that it indeed requires a clever combination of algorithmic techniques to be solved
and that it can be solved using a range of approaches with different solutions having different algorithmic complexities.
Remove problems that do not fit these criteria.

# Step 4.

Write the optimal solution for each of the problems remaining after Step 3 in Python.
Leave only those who can be solved in at most 100 lines of code.

# Step 5.

Come up with a potential storyline for each of the problems remaining after step 4. Assess the interestingness of the storyline
on a scale from 1 (boring) to 5 (exciting).

Leave only the problems that scored at least 4.

# Step 6.

Finally, pick one of the problems from step 5 and:

  - List what are the possible algorithmic solutions with possible algorithmic complexities.
    For each such algorithmic complexity, create a test case (i.e. a pair of input and output files)
    that could be solved using a Python solution with such algorithmic complexity within 0.1 seconds,
    but would not be solvable by code that has worse complexity.

  - List all the tricky corner cases the problem may have and for each corner case create a test case that would fail
    unless the solution code takes that corner case into account.

# Step 7.

Output the produced problem in the following format:

<problem>
<statement>
.. problem statement in Markdown format ...
</statement>
<solutions>
  <solution complexity="linear/quadratic/...">
    ... solution in Python ...
  </solution>
  <solution ...>
    ... solutions for other complexities ...
  </solution>
</solutions>
<tests>
  <test comment="..explanation of the test case">
    <input>
      .. input text data ...
    </input>
    <output>
    .. expected output text data...
    </output>
  </test>
  ... other tests as required
</tests>
""",
    description='Prompt:',
    disabled=False,
    layout=widgets.Layout(height='800px', width='auto'), # Adjust height/width
    style={'description_width': 'initial'}
)
task_author_prompt

Textarea(value='You are an experienced author of informatics olympiad problems.\n\nYour task is to come up wit…

In [ ]:
#@title Generate task
%%time
task_author = genai.GenerativeModel(
  model_name="gemini-2.5-pro-exp-03-25",
  generation_config= {
      "temperature": 0.5,
      "top_p": 1,
      "top_k": 50,
  },
)
generated_task = task_author.generate_content(task_author_prompt.value)

CPU times: user 4.34 s, sys: 569 ms, total: 4.91 s
Wall time: 4min 54s


### Full thinking process (collapsible)

In [ ]:
print(generated_task.text)

Okay, let's create a new informatics olympiad problem step by step.

# Step 1. List Common Algorithms/Techniques

Here's a list of common algorithms and algorithmic techniques used in informatics olympiads:

1.  **Sorting:** Quicksort, Mergesort, Heapsort, Counting Sort, Radix Sort. Comparison-based sorting (`O(N log N)`), linear time sorting (`O(N)` or `O(N+K)`).
2.  **Searching:** Binary Search (on answer, on data), Ternary Search.
3.  **Graph Algorithms:**
    *   Traversal: Breadth-First Search (BFS), Depth-First Search (DFS).
    *   Shortest Paths: Dijkstra's Algorithm, Bellman-Ford, Floyd-Warshall, SPFA (Shortest Path Faster Algorithm).
    *   Minimum Spanning Tree (MST): Prim's Algorithm, Kruskal's Algorithm.
    *   Network Flow: Max Flow (Ford-Fulkerson, Edmonds-Karp, Dinic), Min Cost Max Flow.
    *   Matching: Bipartite Matching.
    *   Topological Sort.
    *   Strongly Connected Components (SCC), Bridges, Articulation Points.
    *   Lowest Common Ancestor (LCA).
4.  **

## Generated problem

In [ ]:
#@title Extract data
# (it's not really parsable XML)
import re
from IPython import display

problem_data = generated_task.text[generated_task.text.index("<problem>")+len("<problem>"):]
problem_data = problem_data[:problem_data.index("</problem>")]
statement = re.findall("<statement>(.+?)</statement>", problem_data, re.DOTALL)[0].strip()
solutions = re.findall("<solution complexity=\"(.+?)\">(.+?)</solution>", problem_data, re.DOTALL)
tests = re.findall("<test comment=\"(.+?)\">.*?<input>(.+?)</input>.*?<output>(.+?)</output>.*?</test>", problem_data, re.DOTALL)
tests = [(cmt, inp.strip(), out.strip()) for cmt, inp, out in tests]

In [ ]:
#@title Problem statement
display.display(display.Markdown(statement))

# Ancient Runes Translation

Archaeologists excavating a historical site near the Baltic Sea have made a fascinating discovery: `N` ancient rune stones, each inscribed with a sequence of symbols. Alongside these stones, they found fragments of a dictionary containing `M` known words believed to belong to the same ancient language.

The archaeologists hypothesize that some rune inscriptions might contain these dictionary words as **subsequences**. A word `w` is a subsequence of a text `s` if `w` can be obtained from `s` by deleting zero or more symbols from `s` without changing the order of the remaining symbols. For example, "ACE" is a subsequence of "ABCDE".

To test their hypothesis, they want to find the maximum number of unique pairs `(rune_stone, dictionary_word)` they can form, such that the dictionary word is a subsequence of the symbols on the rune stone. There are two important constraints:
1. Each rune stone can be part of at most one pair.
2. Each dictionary word can be part of at most one pair.

Your task is to help the archaeologists by calculating this maximum possible number of pairs.

**Input**

The first line contains an integer `N` (0 <= N <= 1000), the number of rune stones.
The next `N` lines each contain a string `s_i`, representing the sequence of symbols on the i-th rune stone (1-based index).
The following line contains an integer `M` (0 <= M <= 1000), the number of dictionary words.
The next `M` lines each contain a string `w_j`, representing the j-th dictionary word (1-based index).

The strings consist of uppercase English letters ('A'-'Z'). The length of each string `s_i` and `w_j` is between 0 and 1000, inclusive.

**Output**

Output a single integer representing the maximum number of pairs `(rune_stone, dictionary_word)` that can be formed according to the rules.

**Constraints**

*   0 <= N <= 1000
*   0 <= M <= 1000
*   0 <= length(`s_i`), length(`w_j`) <= 1000
*   The total number of symbols across all `s_i` is at most 1,000,000.
*   The total number of symbols across all `w_j` is at most 1,000,000.
*   *Note:* The constraints imply that the `O(N*M*L)` part for subsequence checking might be up to `N * M * avg_L`, which could be around `1000 * 1000 * 1 = 10^6` if lengths are small on average, or higher if lengths are concentrated. The total length constraint limits the sum, ensuring the total time for subsequence checks across all pairs is manageable if done efficiently. Total subsequence check time is bounded by Sum(len(s_i)) * M or Sum(len(w_j)) * N, roughly `10^6 * 1000 = 10^9` in worst case if not careful. A better bound is Sum over all pairs (i,j) of O(len(s_i)). Total time for building graph edges: Sum_{i=1..N} Sum_{j=1..M} O(len(s_i)) = Sum_{i=1..N} ( O(len(s_i)) * M ) = O(M * TotalLen(S)). Similarly O(N * TotalLen(W)). So graph building is roughly O(min(M*TotalLen(S), N*TotalLen(W))), which is feasible around `1000 * 10^6 = 10^9`. This is still too high. Let's re-evaluate. The subsequence check is O(len(text)). So building the graph takes Sum_{i=1..N} Sum_{j=1..M} O(len(s_i)). This is O(M * Sum(len(s_i))). Okay, this is feasible: `1000 * 1,000,000 = 10^9`. Still too high for 1 second!

*   *Revised Constraints:* Let's adjust constraints for feasibility.
    *   `N, M <= 500`. Total length `s_i <= 500,000`. Total length `w_j <= 500,000`.
    *   Graph building: `M * TotalLen(S)` -> `500 * 500,000 = 2.5 * 10^8`. Borderline for Python, OK for C++.
    *   Matching: `V=1000`, `E <= 500*500 = 2.5*10^5`. `O(E*sqrt(V))` -> `2.5e5 * sqrt(1000) approx 2.5e5 * 31.6 = 7.9 * 10^6`. Fast. `O(E*V)` -> `2.5e5 * 1000 = 2.5 * 10^8`. Borderline/Slow.
    *   Let's use these revised constraints.

*   **Revised Constraints Section:**
    *   0 <= N <= 500
    *   0 <= M <= 500
    *   0 <= length(`s_i`), length(`w_j`) <= 1000 (Individual length limit still useful)
    *   The total number of symbols across all `s_i` (Sum(len(s_i))) is at most 500,000.
    *   The total number of symbols across all `w_j` (Sum(len(w_j))) is at most 500,000.

**Example 1**

Input:
```
3
ABCDE
AXBCY
AZERTY
3
ACE
AY
XYZ
```

Output:
```
2
```

**Explanation 1**
We can form two pairs:
1. Rune stone "ABCDE" and dictionary word "ACE" ("ACE" is a subsequence of "ABCDE").
2. Rune stone "AXBCY" and dictionary word "AY" ("AY" is a subsequence of "AXBCY").
The third rune stone "AZERTY" does not contain "XYZ" as a subsequence. We cannot use "ACE" or "AY" again. Maximum is 2 pairs.

**Example 2**

Input:
```
2
HELLO
HELLO
1
LO
```

Output:
```
1
```

**Explanation 2**
The word "LO" is a subsequence of both "HELLO" rune stones. However, the word "LO" can only be used in one pair. We can pair ("HELLO" stone 1, "LO") or ("HELLO" stone 2, "LO"). The maximum is 1.

**Example 3**

Input:
```
1

1
ABC
```
*(Note: First rune stone is empty string)*

Output:
```
0
```
**Explanation 3**
The non-empty word "ABC" cannot be a subsequence of the empty rune stone inscription.

**Example 4**

Input:
```
2
ABC

2

A
```
*(Note: Second rune stone and first word are empty strings)*

Output:
```
1
```
**Explanation 4**
The empty word "" is a subsequence of "ABC". Alternatively, the empty word "" is a subsequence of the empty rune stone "". We want to maximize pairs. We can pair (rune "ABC", word "") or (rune "", word ""). Max pairs = 1. (Or pair (rune "ABC", word "A")). Max is 1. Let's take (rune "ABC", word "A").

In [ ]:
#@title Sample solutions
for i, (cpl, code) in enumerate(solutions):
  display.display(display.HTML(f"<h2>Solution #{i} ({cpl})</h2>"))
  display.display(display.Markdown(code))


```python
# This solution uses a standard augmenting path algorithm (DFS based), 
# which is O(E*V) in worst case, but often faster in practice.
# For guaranteed O(E*sqrt(V)), Hopcroft-Karp or Max Flow (Dinic) is needed.
# The provided code might TLE on specific test cases targeting O(E*V) vs O(E*sqrt(V)).

import sys
import collections

# Increase recursion depth for deep DFS in matching
try:
    sys.setrecursionlimit(10000) # N+M could be up to 1000
except: # ReadOnlyFilesystemError or other issues
    pass 

def is_subsequence(word, text):
    iterator = iter(text)
    # Check if all characters in word can be found in iterator in order
    return all(c in iterator for c in word)

# Standard Max Bipartite Matching using DFS for augmenting paths
def max_bipartite_matching(graph, n1, n2):
    match = [-1] * n2 # Stores matching for nodes in the second set (words, index 0..n2-1)
    count = 0
    
    # visited array for DFS, reset for each starting node u
    # visited needs to track nodes visited *within the current DFS call* from a starting node i
    
    adj = graph # Use the provided graph directly

    for i in range(n1): # Iterate through rune nodes 0..n1-1
        visited_dfs = [False] * n1 # Reset visited status for each augmenting path search
        
        def dfs(u):
            visited_dfs[u] = True # Mark rune node u as visited in this DFS path
            for v in adj.get(u, []): # Iterate through words v connected to rune u
                matched_node = match[v] # Node currently matched to word v (-1 if none)
                
                # If word v is unmatched, or if its current match can find an alternative partner
                if matched_node == -1 or (not visited_dfs[matched_node] and dfs(matched_node)):
                    match[v] = u # Match word v with rune u
                    return True # Augmenting path found
            return False # No augmenting path found starting from u through its neighbors

        if dfs(i):
            count += 1 # Increment match count if an augmenting path was found starting from i
            
    return count

def solve():
    N = int(sys.stdin.readline())
    runes = [sys.stdin.readline().strip() for _ in range(N)]
    M = int(sys.stdin.readline())
    words = [sys.stdin.readline().strip() for _ in range(M)]

    if N == 0 or M == 0:
        print(0)
        return

    # Build the bipartite graph: Rune nodes 0..N-1, Word nodes 0..M-1
    # adj[i] contains list of word indices j such that words[j] is subsequence of runes[i]
    adj = collections.defaultdict(list)
    
    # Check all pairs (rune_i, word_j)
    for i in range(N):
        for j in range(M):
            # Optimization: If word is longer than rune text, skip
            if len(words[j]) > len(runes[i]):
                continue
            # Optimization: Empty word is subsequence of any text
            if not words[j]:
                 adj[i].append(j)
                 continue
            # Optimization: Non-empty word cannot be subsequence of empty text
            if not runes[i] and words[j]:
                 continue

            if is_subsequence(words[j], runes[i]):
                adj[i].append(j)

    # Calculate maximum matching
    # n1 = N (number of nodes in first part - runes)
    # n2 = M (number of nodes in second part - words)
    result = max_bipartite_matching(adj, N, M)
    print(result)

solve()
```
  


    ```python
    # The Python solution provided above implements the DFS-based augmenting path
    # algorithm, which has a theoretical worst-case complexity of O(E*V) for the matching part.
    # The overall complexity is O(Sum(len(si)*M) + E*V).
    # This might be too slow if the graph structure triggers the worst-case behavior 
    # of the matching algorithm.
    
    # Paste the same code as above here, as it represents this complexity class.
    import sys
    import collections

    try:
        sys.setrecursionlimit(10000) 
    except:
        pass 

    def is_subsequence(word, text):
        iterator = iter(text)
        return all(c in iterator for c in word)

    def max_bipartite_matching(graph, n1, n2):
        match = [-1] * n2 
        count = 0
        adj = graph 

        for i in range(n1): 
            visited_dfs = [False] * n1 
            
            def dfs(u):
                visited_dfs[u] = True 
                for v in adj.get(u, []): 
                    matched_node = match[v] 
                    if matched_node == -1 or (not visited_dfs[matched_node] and dfs(matched_node)):
                        match[v] = u 
                        return True 
                return False 

            if dfs(i):
                count += 1 
                
        return count

    def solve():
        N = int(sys.stdin.readline())
        runes = [sys.stdin.readline().strip() for _ in range(N)]
        M = int(sys.stdin.readline())
        words = [sys.stdin.readline().strip() for _ in range(M)]

        if N == 0 or M == 0:
            print(0)
            return

        adj = collections.defaultdict(list)
        
        for i in range(N):
            for j in range(M):
                if len(words[j]) > len(runes[i]): continue
                if not words[j]: adj[i].append(j); continue
                if not runes[i] and words[j]: continue
                if is_subsequence(words[j], runes[i]): adj[i].append(j)

        result = max_bipartite_matching(adj, N, M)
        print(result)

    solve()
    ```
  

In [ ]:
#@title Tests
for cmt, inp, out in tests:
  display.display(display.HTML(f"<h3>{cmt}</h3><pre>{inp}</pre><h4>Out</h4><pre>{out}</pre>"))

## Test sample solutions

In [ ]:
#@title Testing code
import textwrap

def test_solution(sol, inp):
  try:
    with open("/tmp/sol.py", "w") as f:
      f.write(sol)
    with open("/tmp/test.in", "w") as f:
      f.write(inp)
    !cat /tmp/test.in | python /tmp/sol.py > /tmp/test.out
    with open("/tmp/test.out") as f:
      return f.read().strip()
  except:
    return "Failed"

def eval_solution(sol_cmt, sol, tests):
  display.display(display.HTML(f"<h2>Testing solution: {sol_cmt}</h2>"))
  sol = sol.strip().replace("```python", "").replace("```", "")
  sol = textwrap.dedent(sol)
  table = ["<table><tr><th>Test</th><th>Input</th><th>Exp. out</th><th>Act. out</th></tr>"]
  for cmt, inp, out in tests:
    test_out = test_solution(sol, inp)
    table.append(f"<tr style='background-color: {'lightgreen' if out == test_out else '#ffaaaa'}'><td>{cmt}</td><td>{inp}</td><td>{out}</td><td>{test_out}</td></tr>")
  table.append("</table>")
  display.display(display.HTML("".join(table)))

for sol_cmt, sol in solutions:
  eval_solution(sol_cmt, sol, tests)

Traceback (most recent call last):
  File "/tmp/sol.py", line 87, in <module>
    solve()
  File "/tmp/sol.py", line 51, in solve
    N = int(sys.stdin.readline())
        ^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '# Generated by script below (N=500, M=500, Lr=10, Lw=5)\n'
Traceback (most recent call last):
  File "/tmp/sol.py", line 87, in <module>
    solve()
  File "/tmp/sol.py", line 51, in solve
    N = int(sys.stdin.readline())
        ^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '# Generated by script below (N=100, M=100, L=1000)'


Test,Input,Exp. out,Act. out
Example 1: Basic case,3 ABCDE AXBCY AZERTY 3 ACE AY XYZ,2,2
"Example 2: Identical runes, one word",2 HELLO HELLO 1 LO,1,1
Example 3: Empty rune text,1 1 ABC,0,0
Example 4: Empty word and empty rune text,2 ABC 2 A,1,2
Corner Case: N=0,0 3 WORDONE WORDTWO WORDTHREE,0,0
Corner Case: M=0,3 RUNEONE RUNETWO RUNETHREE 0,0,0
Corner Case: Empty word matches non-empty text,1 ABC 1,1,1
Corner Case: Empty word matches empty text,1 1,1,1
"Corner Case: Identical words, one rune",1 ABCDE 2 ACE ACE,1,1
Corner Case: No matches possible,2 ABC DEF 2 XYZ PQR,0,0


Traceback (most recent call last):
  File "/tmp/sol.py", line 65, in <module>
    solve()
  File "/tmp/sol.py", line 44, in solve
    N = int(sys.stdin.readline())
        ^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '# Generated by script below (N=500, M=500, Lr=10, Lw=5)\n'
Traceback (most recent call last):
  File "/tmp/sol.py", line 65, in <module>
    solve()
  File "/tmp/sol.py", line 44, in solve
    N = int(sys.stdin.readline())
        ^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '# Generated by script below (N=100, M=100, L=1000)'


Test,Input,Exp. out,Act. out
Example 1: Basic case,3 ABCDE AXBCY AZERTY 3 ACE AY XYZ,2,2
"Example 2: Identical runes, one word",2 HELLO HELLO 1 LO,1,1
Example 3: Empty rune text,1 1 ABC,0,0
Example 4: Empty word and empty rune text,2 ABC 2 A,1,2
Corner Case: N=0,0 3 WORDONE WORDTWO WORDTHREE,0,0
Corner Case: M=0,3 RUNEONE RUNETWO RUNETHREE 0,0,0
Corner Case: Empty word matches non-empty text,1 ABC 1,1,1
Corner Case: Empty word matches empty text,1 1,1,1
"Corner Case: Identical words, one rune",1 ABCDE 2 ACE ACE,1,1
Corner Case: No matches possible,2 ABC DEF 2 XYZ PQR,0,0


# Part 2: AI solves the task

In [ ]:
#@title Prompt
task_solver_prompt = widgets.Textarea(
    value="""You are a world-class competitive programming sportsman.

You will be given the problem statement for an olympiad problem and will need to
come up with a Python program that solves this problem. Your program will
have to read input data from stdin and write the answer to stdout.

Please, proceed step by step:

Step 1.
Analyze the provided problem, and propose an algorithmic approach for solving it.
Assess the algorithmic time and space complexity of your solution.

Step 2.
Write the Python code for your solution.

Step 3.
Make sure your code covers all corner cases. If it doesn't, fix it.

Step 4.
Output the Python code for your solution.

Make sure to wrap it with <solution> tags as follows:

<solution>
.. Python solution code ...
</solution>

Make sure your code is well commented and clear.
""",
    description='Prompt:',
    disabled=False,
    layout=widgets.Layout(height='400px', width='auto'), # Adjust height/width
    style={'description_width': 'initial'}
)
task_solver_prompt

Textarea(value="You are a world-class competitive programming sportsman.\n\nYou will be given the problem stat…

In [ ]:
#@title Solve problem
%%time
task_solver = genai.GenerativeModel(
  model_name="gemini-2.5-pro-exp-03-25",
  generation_config= {
      "temperature": 0.5,
      "top_p": 1,
      "top_k": 50,
  },
)
solved_task = task_solver.generate_content(task_solver_prompt.value + """

PROBLEM STATEMENT
""" + statement)

CPU times: user 2.91 s, sys: 308 ms, total: 3.21 s
Wall time: 2min 41s


### Full thinking process (collapsible)

In [ ]:
print(solved_task.text)

```python
import sys

# Step 1: Analyze the problem and propose an algorithmic approach

# The problem asks for the maximum number of pairs (rune_stone, dictionary_word) such that
# the word is a subsequence of the stone's inscription, with the constraint that each
# stone and each word can be used in at most one pair. This is a maximum bipartite matching problem.

# We can model this problem using a bipartite graph G = (U U V, E), where:
# - U is the set of N vertices representing the rune stones (u_1, ..., u_N).
# - V is the set of M vertices representing the dictionary words (v_1, ..., v_M).
# - An edge (u_i, v_j) exists in E if and only if the dictionary word w_j is a subsequence of the rune stone inscription s_i.

# The goal is to find a matching in this graph with the maximum possible number of edges.
# A matching is a subset of edges where no two edges share a common vertex.

# The algorithm steps are:
# 1. Construct the bipartite graph:
#    - Iterate through all pairs of rune 

## Solution evaluation

In [ ]:
#@title Extract data
# (it's not really parsable XML)
import re
from IPython import display


if "<solution>" in solved_task.text:
  solution = textwrap.dedent(re.findall("<solution>(.+?)</solution>", solved_task.text, re.DOTALL)[0]).strip()
else:
  solution = solved_task.text
solution = solution.replace("```python", "").replace("```", "")


In [ ]:
print(solution)


import sys

# Step 1: Analyze the problem and propose an algorithmic approach

# The problem asks for the maximum number of pairs (rune_stone, dictionary_word) such that
# the word is a subsequence of the stone's inscription, with the constraint that each
# stone and each word can be used in at most one pair. This is a maximum bipartite matching problem.

# We can model this problem using a bipartite graph G = (U U V, E), where:
# - U is the set of N vertices representing the rune stones (u_1, ..., u_N).
# - V is the set of M vertices representing the dictionary words (v_1, ..., v_M).
# - An edge (u_i, v_j) exists in E if and only if the dictionary word w_j is a subsequence of the rune stone inscription s_i.

# The goal is to find a matching in this graph with the maximum possible number of edges.
# A matching is a subset of edges where no two edges share a common vertex.

# The algorithm steps are:
# 1. Construct the bipartite graph:
#    - Iterate through all pairs of rune stones (i

In [ ]:
eval_solution("Proposed solution", solution, tests)

Traceback (most recent call last):
  File "/tmp/sol.py", line 247, in <module>
    solve()
  File "/tmp/sol.py", line 170, in solve
    N = int(sys.stdin.readline())
        ^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '# Generated by script below (N=500, M=500, Lr=10, Lw=5)\n'
Traceback (most recent call last):
  File "/tmp/sol.py", line 247, in <module>
    solve()
  File "/tmp/sol.py", line 170, in solve
    N = int(sys.stdin.readline())
        ^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: '# Generated by script below (N=100, M=100, L=1000)'


Test,Input,Exp. out,Act. out
Example 1: Basic case,3 ABCDE AXBCY AZERTY 3 ACE AY XYZ,2,2
"Example 2: Identical runes, one word",2 HELLO HELLO 1 LO,1,1
Example 3: Empty rune text,1 1 ABC,0,0
Example 4: Empty word and empty rune text,2 ABC 2 A,1,2
Corner Case: N=0,0 3 WORDONE WORDTWO WORDTHREE,0,0
Corner Case: M=0,3 RUNEONE RUNETWO RUNETHREE 0,0,0
Corner Case: Empty word matches non-empty text,1 ABC 1,1,1
Corner Case: Empty word matches empty text,1 1,1,1
"Corner Case: Identical words, one rune",1 ABCDE 2 ACE ACE,1,1
Corner Case: No matches possible,2 ABC DEF 2 XYZ PQR,0,0
